In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 2: install libs
!pip install --upgrade pip
!pip install timm pandas openpyxl scikit-learn pillow tqdm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
import pandas as pd

EXCEL_PATH = "/content/drive/MyDrive/Copy of final_herbify_dataset.xlsx"

df = pd.read_excel(EXCEL_PATH, engine="openpyxl")

print("✅ Loaded Excel successfully!")
print("Columns:", df.columns.tolist())
print(df.head())



✅ Loaded Excel successfully!
Columns: ['image_path', 'scientific_name', 'local_name', 'medicinal_uses', 'parts_used', 'Other Uses', 'Cultivation Details']
                                          image_path scientific_name  \
0  /content/drive/MyDrive/herb_dataset/Herbify-Da...     allium cepa   
1  /content/drive/MyDrive/herb_dataset/Herbify-Da...     allium cepa   
2  /content/drive/MyDrive/herb_dataset/Herbify-Da...     allium cepa   
3  /content/drive/MyDrive/herb_dataset/Herbify-Da...     allium cepa   
4  /content/drive/MyDrive/herb_dataset/Herbify-Da...     allium cepa   

  local_name                                     medicinal_uses parts_used  \
0      Onion  used to heal abscesses, remove intestinal worm...       Bulb   
1      Onion  used to heal abscesses, remove intestinal worm...       Bulb   
2      Onion  used to heal abscesses, remove intestinal worm...       Bulb   
3      Onion  used to heal abscesses, remove intestinal worm...       Bulb   
4      Onion  used to 

In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/herb_dataset.zip"
extract_dir = "/content/drive/MyDrive/herb_dataset"

# unzip only if not already extracted
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print("✅ Unzipped to:", extract_dir)

# check first 10 folders
!ls "/content/drive/MyDrive/herb_dataset" | head


✅ Unzipped to: /content/drive/MyDrive/herb_dataset
Herbify-Dataset


In [ ]:
!ls "/content/drive/MyDrive/herb_dataset/"


Herbify-Dataset


In [ ]:
missing = [p for p in df["image_path"].tolist() if not os.path.exists(p)]
print(f"Total images: {len(df)}")
print(f"Missing images: {len(missing)}")


Total images: 6058
Missing images: 0


In [ ]:
!pip install transformers datasets timm


In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
from datasets import Dataset as HFDataset

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


In [ ]:
import numpy as np

def load_example(row):
    image = Image.open(row["image_path"]).convert("RGB").resize((224,224))
    image = transform(image)               # torch.Tensor [3,224,224]
    return {
        "pixel_values": image.numpy(),     # convert to numpy
        "labels": label2id[row["scientific_name"]]
    }


In [ ]:
# clean scientific_name column
df = df.dropna(subset=["scientific_name"])  # remove rows with missing labels
df["scientific_name"] = df["scientific_name"].astype(str).str.strip().str.lower()

# map scientific names to labels
labels = sorted(df["scientific_name"].unique())
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for i, l in enumerate(labels)}

print(f"✅ Total classes: {len(labels)}")
print("Example classes:", labels[:10])


✅ Total classes: 89
Example classes: ['allium cepa', 'aloe barbadensis miller', 'alpinia galanga', 'amaranthus viridis', 'andrographis paniculata', 'annona squamosa', 'artocarpus heterophyllus', 'azadirachta indica', 'bacopa monnieri', 'bambusa vulgaris']


In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True   # 👈 this fixes it
)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([89]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([89, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install -U transformers


In [ ]:
# run this cell first
!pip install -q timm


In [ ]:
# Run this whole cell after the install cell
import os, random, numpy as np, pandas as pd, torch
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import timm
from torch import nn
from tqdm import tqdm

# ----- CONFIG -----
EXCEL_PATH = "/content/drive/MyDrive/Copy of final_herbify_dataset.xlsx"  # adjust if needed
IMAGE_COL = "image_path"
LABEL_COL = "scientific_name"
OUTPUT_DIR = "/content/drive/MyDrive/vit_medicinal_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE = 16   # lower if you hit OOM
NUM_EPOCHS = 5
LR = 3e-5
IMG_SIZE = 224
MODEL_NAME = "vit_base_patch16_224"   # timm model name
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 4

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

print("Device:", DEVICE)

# ----- LOAD & CLEAN DATAFRAME -----
df = pd.read_excel(EXCEL_PATH, engine="openpyxl")
# drop missing images/labels
df = df.dropna(subset=[IMAGE_COL, LABEL_COL]).copy()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip().str.lower()
print("Total rows after dropna:", len(df))
classes = sorted(df[LABEL_COL].unique())
print("Num classes:", len(classes))

label2id = {l: i for i, l in enumerate(classes)}
id2label = {i: l for i, l in enumerate(classes)}

# ----- DATASET -----
class HerbifyDataset(Dataset):
    def __init__(self, df, image_col, label_col, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_col = image_col
        self.label_col = label_col
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row[self.image_col]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = label2id[row[self.label_col]]
        return img, label

# ----- TRANSFORMS -----
train_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomRotation(20),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

val_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# ----- SPLIT -----
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df[LABEL_COL], random_state=SEED)
print("Train rows:", len(train_df), "Val rows:", len(val_df))

train_ds = HerbifyDataset(train_df, IMAGE_COL, LABEL_COL, transform=train_tfms)
val_ds = HerbifyDataset(val_df, IMAGE_COL, LABEL_COL, transform=val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# ----- MODEL -----
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=len(classes))
model.to(DEVICE)

# ----- OPTIMIZER / LOSS -----
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()


# ----- TRAIN LOOP -----
best_val_acc = 0.0
for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} (train)")
    for imgs, labels in pbar:
        imgs = imgs.to(DEVICE, non_blocking=True)
        labels = labels.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)
        pbar.set_postfix(train_loss=(running_loss/total), train_acc=(correct/total))

    train_loss = running_loss / total
    train_acc = correct / total

    # validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(DEVICE, non_blocking=True)
            labels = labels.to(DEVICE, non_blocking=True)
            logits = model(imgs)
            loss = criterion(logits, labels)
            val_loss += loss.item() * imgs.size(0)
            preds = logits.argmax(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += imgs.size(0)

    val_loss = val_loss / val_total
    val_acc = val_correct / val_total

    print(f"Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.4f}  |  val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

    # save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_dict = {
            "model_state_dict": model.state_dict(),
            "id2label": id2label,
            "label2id": label2id,
            "model_name": MODEL_NAME
        }
        torch.save(save_dict, os.path.join(OUTPUT_DIR, "best_vit.pth"))
        print(f"Saved best model (val_acc={val_acc:.4f}) to {OUTPUT_DIR}/best_vit.pth")

print("Training finished. Best val acc:", best_val_acc)


Device: cuda
Total rows after dropna: 6057
Num classes: 89
Train rows: 5148 Val rows: 909


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/5 (train): 100%|██████████| 322/322 [03:37<00:00,  1.48it/s, train_acc=0.637, train_loss=1.56]


Epoch 1: train_loss=1.5570, train_acc=0.6369  |  val_loss=0.2733, val_acc=0.9230
Saved best model (val_acc=0.9230) to /content/drive/MyDrive/vit_medicinal_output/best_vit.pth


Epoch 2/5 (train): 100%|██████████| 322/322 [03:11<00:00,  1.68it/s, train_acc=0.953, train_loss=0.165]


Epoch 2: train_loss=0.1646, train_acc=0.9526  |  val_loss=0.2153, val_acc=0.9263
Saved best model (val_acc=0.9263) to /content/drive/MyDrive/vit_medicinal_output/best_vit.pth


...

Epoch 3: train_loss=0.0804, train_acc=0.9800  |  val_loss=0.1204, val_acc=0.9703
Saved best model (val_acc=0.9703) to /content/drive/MyDrive/vit_medicinal_output/best_vit.pth


Epoch 4/5 (train): 100%|██████████| 322/322 [03:01<00:00,  1.77it/s, train_acc=0.985, train_loss=0.0554]


Epoch 4: train_loss=0.0554, train_acc=0.9847  |  val_loss=0.1605, val_acc=0.9615


Epoch 5/5 (train): 100%|██████████| 322/322 [02:52<00:00,  1.86it/s, train_acc=0.985, train_loss=0.0583]


Epoch 5: train_loss=0.0583, train_acc=0.9854  |  val_loss=0.1678, val_acc=0.9626
Training finished. Best val acc: 0.9702970297029703


In [ ]:
import torch, timm
import pandas as pd
from PIL import Image
import torchvision.transforms as T

# Paths
MODEL_PATH = "/content/drive/MyDrive/vit_medicinal_output/best_vit.pth"
EXCEL_PATH = "/content/drive/MyDrive/Copy of final_herbify_dataset.xlsx"
LABEL_COL = "scientific_name"
IMG_SIZE = 224
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Reload dataset info
df_all = pd.read_excel(EXCEL_PATH, engine="openpyxl")
df_all[LABEL_COL] = df_all[LABEL_COL].astype(str).str.strip().str.lower()

# Load checkpoint
ckpt = torch.load(MODEL_PATH, map_location=DEVICE)
model = timm.create_model(
    ckpt.get("model_name", "vit_base_patch16_224"),
    pretrained=False,
    num_classes=len(ckpt["label2id"])
)
model.load_state_dict(ckpt["model_state_dict"])
model.to(DEVICE)
model.eval()

id2label = {int(k): v for k, v in ckpt["id2label"].items()}  # ensure int keys

# Preprocessing transform
tfm = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# Prediction helper
def predict_and_lookup(image_path, topk=3):
    img = Image.open(image_path).convert("RGB")
    x = tfm(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
    topk_inds = probs.argsort()[::-1][:topk]
    results = []
    for idx in topk_inds:
        species = id2label[int(idx)]
        score = float(probs[idx])
        rows = df_all[df_all[LABEL_COL].astype(str).str.strip() == species].to_dict(orient="records")
        results.append({
            "predicted_species": species,
            "score": round(score, 4),
            "details": rows[:1]  # first matching row from Excel
        })
    return results


In [ ]:
# Build lookup dictionary once after loading df
plant_data = {}

for _, row in df.iterrows():
    sci_name = row["scientific_name"]
    plant_data[sci_name] = {
        "scientific_name": row.get("scientific_name", "N/A"),
        "local_name": row.get("local_name", "N/A"),
        "medicinal_uses": row.get("medicinal_uses", "N/A"),
        "parts_used": row.get("parts_used", "N/A"),
        "Other Uses": row.get("Other Uses", "N/A"),
        "Cultivation Details": row.get("Cultivation Details", "N/A")
    }


In [ ]:
import torch
from PIL import Image
import torch.nn.functional as F

# Make sure model is on correct device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()


def predict_and_lookup_fn(image_path, topk=1):
    image = Image.open(image_path).convert("RGB").resize((224,224))
    inputs = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(inputs)
        probs = torch.nn.functional.softmax(outputs, dim=-1)[0]


    values, indices = torch.topk(probs, k=topk)

    values = values.tolist()
    indices = indices.tolist()
    if isinstance(values, float):
        values = [values]
        indices = [indices]

    results = []
    for score, idx in zip(values, indices):
        label = id2label[idx]
        details = plant_data.get(label, {})

        # Format as ordered string
        formatted = f"""
Scientific Name: {details.get("scientific_name", label)}
Local Name: {details.get("local_name", "N/A")}
Medicinal Uses: {details.get("medicinal_uses", "N/A")}
Parts Used: {details.get("parts_used", "N/A")}
Other Uses: {details.get("Other Uses", "N/A")}
Cultivation Details: {details.get("Cultivation Details", "N/A")}
Prediction Score: {round(score, 4)}
"""
        results.append(formatted.strip())

    return "\n\n".join(results)



In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    filepath = f"./{filename}"   # path to uploaded file
    print(f"Uploaded file: {filepath}")
    results = predict_and_lookup_fn(filepath, topk=1)
    from pprint import pprint
    pprint(results)


Saving 596.jpg to 596.jpg
Uploaded file: ./596.jpg
('Scientific Name: mentha\n'
 'Local Name: Mint\n'
 'Medicinal Uses: treat gallbladder and stomach diseases, cholera, digestive '
 'disorders, headache, vomiting, as an antiemetic, emetic, astringent, and as '
 'a general tonic for rejuvenation.\n'
 'Parts Used: Leaf\n'
 'Other Uses: Repellent Strewing Agroforestry uses: Mint can be used as a '
 'ground cover, helping to suppress weeds and prevent soil erosion. Its strong '
 'aroma can also help deter pests. The plant repels flies, mice and rats[14, '
 '20, 244]. It has a pleasant, fresh scent and was formerly used as a strewing '
 'herb and has been strewn in granaries to keep mice and rats off the '
 'grain[14, 244]. The plant, harvested before flowering, yields about 0.8% '
 'essential oil[240]. The fresh or dried plant is very good when used in '
 'herbal baths and can also be used in herb pillows[244]. Water garden. 1. '
 'Nectary - Flowers rich in nectar and pollen: Yes – Mint sp

In [ ]:
df.isna().sum()


,0
image_path,0
scientific_name,0
local_name,0
medicinal_uses,0
parts_used,0
Other Uses,0
Cultivation Details,0


In [ ]:
def compute_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

train_accuracy = compute_accuracy(train_loader, model, DEVICE)
val_accuracy = compute_accuracy(val_loader, model, DEVICE)

print(f"Final Train Accuracy: {train_accuracy:.4f}")
print(f"Final Validation Accuracy: {val_accuracy:.4f}")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Final Train Accuracy: 0.9891
Final Validation Accuracy: 0.9703


In [ ]:
import os

for item in os.listdir():
    print(item)


.config
drive
sample_data


In [ ]:
import os

path = "/content/drive/MyDrive"
for item in os.listdir(path):
    print(item)


Colab Notebooks
herb_dataset.zip
herb_dataset
vit_medicinal_output
Copy of Untitled1.ipynb
Copy of final_herbify_dataset (1).gsheet
Copy of final_herbify_dataset.gsheet
Copy of final_herbify_dataset.xlsx


In [ ]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/Copy of final_herbify_dataset.xlsx")

df["scientific_name"] = df["scientific_name"].astype(str).str.strip().str.lower()

unique_plants = df["scientific_name"].nunique()

print("Number of unique medicinal plants:", unique_plants)


Number of unique medicinal plants: 90


In [ ]:
import os

folder_path = "/content/drive/MyDrive/herb_dataset"

image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".webp"]

count = 0
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if any(file.lower().endswith(ext) for ext in image_extensions):
            count += 1

print("Total number of images:", count)


Total number of images: 6104
